In [10]:
import cv2
from __future__ import print_function

import numpy as np
import keras

from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Add, concatenate, AveragePooling2D, Lambda
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K
from keras import activations
import os
from keras.models import load_model
from matplotlib import pyplot as plt
from random import shuffle
import pandas as pd
import tensorflow as tf

In [2]:
random_seed=305


np.random.seed(random_seed)
tf.set_random_seed(random_seed)

In [3]:
# 이미지 데이터 불러오기 train을 train / valid로 랜덤으로 나누어 줍니다.

defect_class = "BIN_Crop"

modelname = defect_class + "_Single_BIN_Crop.hdf5."
train_path = './Data/BIN_CROP_train.npz'
test_path = './Data/BIN_CROP_test.npz'

saved_trainset = np.load(train_path)
saved_testset = np.load(test_path)

X_train_data = saved_trainset['x']
train_data_y = saved_trainset['y']

X_train_data = X_train_data/255

n_data = X_train_data.shape[0]
indices = np.arange(n_data)
np.random.shuffle(indices)

train_index = indices[int(n_data * 0.2):n_data+1]
valid_index = indices[0:int(n_data * 0.2)]

X_train = X_train_data[train_index]
X_valid = X_train_data[valid_index]
train_y = train_data_y[train_index]
valid_y = train_data_y[valid_index]



X_test = saved_testset['x']
test_y = saved_testset['y']

X_test=X_test/255

print(X_train.shape)
print(train_y.shape)
print(X_test.shape)
print(test_y.shape)
print(X_valid.shape)
print(valid_y.shape)

(1920, 256, 256, 1)
(1920, 4)
(476, 256, 256, 1)
(476, 4)
(480, 256, 256, 1)
(480, 4)


In [4]:
# 결함정보 불러오기
import pandas as pd

train_csv_path = './scaled_train_defect_feature.csv'
test_csv_path = './scaled_train_defect_feature.csv'


train_wide_data = pd.read_csv(train_csv_path, header = None)

train_wide = np.array(train_wide_data)[train_index,1:]
valid_wide = np.array(train_wide_data)[valid_index,1:]

test_wide = pd.read_csv(test_csv_path, header = None) 

print(np.shape(train_wide))

(1920, 21)


In [5]:
#Data Auㅎmentation은 이미지에만 적용하도록 하는 함수

train_datagen = ImageDataGenerator(
    zoom_range=0.3, 
    horizontal_flip=True, 
    vertical_flip=True,
    rotation_range=90) 

test_datagen = ImageDataGenerator()


def train_gen_flow_for_two_inputs(X1, X2, y):
    genX1 = train_datagen.flow(X1,y,  batch_size=64,shuffle=False)
    genX2 = train_datagen.flow(X1,X2, batch_size=64,shuffle=False)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            yield [X1i[0], X2i[1]], X1i[1]
            

def test_gen_flow_for_two_inputs(X1, X2, y):
    genX1 = test_datagen.flow(X1,y,  batch_size=64,shuffle=False)
    genX2 = test_datagen.flow(X1,X2, batch_size=64,shuffle=False)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            yield [X1i[0], X2i[1]], X1i[1]
            

train_generator = train_gen_flow_for_two_inputs(X_train, train_wide, train_y)

valid_generator = test_gen_flow_for_two_inputs(X_valid, valid_wide, valid_y)


In [6]:
img = Input(shape=(256, 256, 1),name="deep")
defect_info = Input(shape=(21,),name="wide")


VGG=Conv2D(32,(7,7),padding='same',activation='relu', strides = 2)(img)
VGG=Conv2D(32,(3,3),padding='same',activation='relu')(VGG)
VGG=MaxPooling2D(pool_size=(2, 2))(VGG)
VGG=Dropout(0.25)(VGG)

VGG=Conv2D(64,(3,3),padding='same',activation='relu')(VGG)
VGG=Conv2D(64,(3,3),padding='same',activation='relu')(VGG)
VGG=MaxPooling2D(pool_size=(2, 2))(VGG)
VGG=Dropout(0.25)(VGG)


VGG=Conv2D(128,(3,3),padding='same',activation='relu')(VGG)
VGG=Conv2D(128,(3,3),padding='same',activation='relu')(VGG)
VGG=MaxPooling2D(pool_size=(2, 2))(VGG)
VGG=Dropout(0.25)(VGG)

VGG=Conv2D(256,(3,3),padding='same',activation='relu')(VGG)
VGG=Conv2D(256,(3,3),padding='same',activation='relu')(VGG)
VGG=Conv2D(256,(3,3),padding='same',activation='relu')(VGG)
VGG=MaxPooling2D(pool_size=(2, 2))(VGG)
VGG=Dropout(0.25)(VGG)
           
VGG=Conv2D(512,(3,3),padding='same',activation='relu')(VGG)
VGG=Conv2D(512,(3,3),padding='same',activation='relu')(VGG)
VGG=Conv2D(512,(3,3),padding='same',activation='relu')(VGG)
   
VGG=GlobalAveragePooling2D(name='VGG_GMP')(VGG)
VGG=Dropout(0.25)(VGG)


wide = Dropout(0.5)(defect_info)
Subnet=concatenate([VGG,wide],axis=-1)
Sub1=Dense(16,activation='relu')(Subnet)
dense=Dense(4,activation='softmax', name='preds')(Sub1)

model=Model(inputs=[img,defect_info],outputs=dense)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep (InputLayer)               (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 32) 1600        deep[0][0]                       
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 32) 9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 64, 64, 32)   0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
dropout_1 

In [7]:
sgd = SGD(lr=0.001, decay=1e-5, momentum=0.9, nesterov=True)
# adamopt = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=sgd,  metrics=['accuracy'])

In [8]:
# hist = model.fit_generator(
#         train_generator,
#         steps_per_epoch=50,
#         epochs=400,
#         verbose=1,
#         validation_data=valid_generator,
#         validation_steps=30, 
#         callbacks=None)


# fig, loss_ax = plt.subplots()

# acc_ax = loss_ax.twinx()

# loss_ax.plot(hist.history['loss'], 'y', label='train loss')
# loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

# acc_ax.plot(hist.history['acc'], 'b', label='train acc')
# acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')

# loss_ax.set_xlabel('epoch')
# loss_ax.set_ylabel('loss')
# acc_ax.set_ylabel('accuracy')

# plt.show()

In [9]:
# model.save(modelname)